In [ ]:
# =====================================================
# FAKE NEWS DETECTION – (ALBERT-base-v2) + WELFAKE
# =====================================================

# 1. INSTALL & IMPORT
!pip install -q transformers datasets torch scikit-learn pandas numpy psutil accelerate

import os, re, shutil, psutil, warnings
import pandas as pd
import numpy as np
import torch
from datasets import load_dataset, Dataset, DatasetDict
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_recall_fscore_support, roc_auc_score
from sklearn.utils.class_weight import compute_class_weight
from transformers import (
    AutoTokenizer, AutoModelForSequenceClassification,
    Trainer, TrainingArguments, EarlyStoppingCallback,
    DataCollatorWithPadding
)
from google.colab import drive
from tqdm.auto import tqdm

warnings.filterwarnings("ignore")

# 2. GPU INFO
if torch.cuda.is_available():
    device_name = torch.cuda.get_device_name(0)
    vram_gb = torch.cuda.get_device_properties(0).total_memory / 1e9
else:
    device_name = "CPU"
    vram_gb = 0.0

print(f"Device: {device_name} | CUDA: {torch.cuda.is_available()} | VRAM: {vram_gb:.1f} GB")

# 3. MOUNT DRIVE
drive.mount('/content/drive', force_remount=False)

def check_drive_space(path="/content/drive/MyDrive"):
    usage = psutil.disk_usage(path)
    used_gb = usage.used / 1e9
    total_gb = usage.total / 1e9
    pct = used_gb / total_gb * 100
    print(f"Drive: {used_gb:.1f}GB / {total_gb:.1f}GB ({pct:.1f}%)")
    if pct > 90:
        print("⚠️  CẢNH BÁO: Drive gần đầy!")

check_drive_space()

# 4. OUTPUT_DIR + CHECKPOINT MANAGEMENT
OUTPUT_DIR = "/content/drive/MyDrive/WELFake_ALBERT_base_Pro"
os.makedirs(OUTPUT_DIR, exist_ok=True)

def manage_checkpoints(output_dir, keep_latest=2):
    ckpts = [c for c in os.listdir(output_dir) if c.startswith("checkpoint-")]
    if len(ckpts) <= keep_latest:
        return

    def idx(n):
        try:
            return int(n.split("-")[-1])
        except:
            return os.path.getmtime(os.path.join(output_dir, n))

    ckpts_sorted = sorted(ckpts, key=idx)
    for ck in ckpts_sorted[:-keep_latest]:
        shutil.rmtree(os.path.join(output_dir, ck), ignore_errors=True)
        print(f"Đã xóa checkpoint cũ:", ck)

def get_last_checkpoint(output_dir):
    ckpts = [c for c in os.listdir(output_dir) if c.startswith("checkpoint-")]
    if not ckpts:
        return None

    def idx(n):
        try:
            return int(n.split("-")[-1])
        except:
            return os.path.getmtime(os.path.join(output_dir, n))

    ckpts_sorted = sorted(ckpts, key=idx, reverse=True)
    manage_checkpoints(output_dir, keep_latest=2)
    return os.path.join(output_dir, ckpts_sorted[0])

last_checkpoint = get_last_checkpoint(OUTPUT_DIR)
print("Checkpoint gần nhất:", last_checkpoint or "Không có → train từ đầu")

# 5. LOAD & CLEAN DATA
print("\nTải dataset WELFake...")
dataset = load_dataset("davanstrien/WELFake")
df = pd.DataFrame(dataset["train"])
print("Số mẫu ban đầu:", len(df))

df['content'] = df.get('title', '').fillna('') + " [SEP] " + df.get('text', '').fillna('')

def clean_text(s):
    if not isinstance(s, str): return ""
    s = s.lower()
    s = re.sub(r'https?://\S+', ' ', s)
    s = re.sub(r'<.*?>', ' ', s)
    s = re.sub(r'[^a-z0-9\s]', ' ', s)
    s = re.sub(r'\s+', ' ', s).strip()
    return s

df['content'] = df['content'].apply(clean_text)
df = df[df['content'].str.len() > 20].drop_duplicates(subset=['content'])

print("Sau xử lý:", len(df))

label_counts = df['label'].value_counts(normalize=True)
print(f"Fake={label_counts.get(0,0):.1%}, Real={label_counts.get(1,0):.1%}")

# Class weights
classes = np.array([0,1])
class_weights = compute_class_weight('balanced', classes=classes, y=df['label'])
print("Class weights:", class_weights)

# 6. SPLIT
labels = df['label'].values
train_df, temp_df = train_test_split(df, test_size=0.25, random_state=42, stratify=labels)
val_df, test_df = train_test_split(temp_df, test_size=0.5, random_state=42, stratify=temp_df['label'])

print(f"Train={len(train_df)}, Val={len(val_df)}, Test={len(test_df)}")

train_dataset = Dataset.from_pandas(train_df[['content','label']].reset_index(drop=True))
val_dataset   = Dataset.from_pandas(val_df[['content','label']].reset_index(drop=True))
test_dataset  = Dataset.from_pandas(test_df[['content','label']].reset_index(drop=True))
dataset_dict = DatasetDict({"train": train_dataset, "validation": val_dataset, "test": test_dataset})

# 7. TOKENIZER (ALBERT)
MODEL_NAME = "albert-base-v2"
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

def tokenize_fn(batch):
    return tokenizer(batch["content"], truncation=True, max_length=384, padding=False)

print("Tokenizing...")
tokenized = dataset_dict.map(tokenize_fn, batched=True, batch_size=1000, remove_columns=['content'])
tokenized = tokenized.rename_column("label", "labels")
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

# 8. MODEL (ALBERT-base)
model = AutoModelForSequenceClassification.from_pretrained(
    MODEL_NAME,
    num_labels=2
)

model.config.id2label = {0:"Fake", 1:"Real"}
model.config.label2id = {"Fake":0, "Real":1}

# 9. TRAINING ARGS
training_args = TrainingArguments(
    output_dir=OUTPUT_DIR,
    num_train_epochs=3,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=16,
    gradient_accumulation_steps=2,
    warmup_ratio=0.1,
    weight_decay=0.01,
    learning_rate=3e-5,
    lr_scheduler_type="linear",
    eval_strategy="epoch",
    save_strategy="epoch",
    save_total_limit=2,
    load_best_model_at_end=True,
    metric_for_best_model="f1",
    greater_is_better=True,
    fp16=torch.cuda.is_available(),
    report_to="none"
)

# 10. METRICS
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    preds = np.argmax(logits, axis=1)

    precision, recall, f1, _ = precision_recall_fscore_support(
        labels, preds, average="weighted", zero_division=0
    )
    acc = accuracy_score(labels, preds)

    try:
        probs = torch.softmax(torch.tensor(logits), dim=1).numpy()[:,1]
        auc = roc_auc_score(labels, probs)
    except:
        auc = None

    return {
        "accuracy": float(acc),
        "precision": float(precision),
        "recall": float(recall),
        "f1": float(f1),
        "auc": auc
    }

# 11. CUSTOM TRAINER with CLASS WEIGHT (fixed signature)
class WeightedTrainer(Trainer):
    def compute_loss(self, model, inputs, return_outputs=False, num_items_in_batch=None):
        labels = inputs.pop("labels")
        outputs = model(**inputs)
        logits = outputs.logits
        w = torch.tensor(class_weights, dtype=torch.float32, device=model.device)
        loss_fct = torch.nn.CrossEntropyLoss(weight=w)
        loss = loss_fct(logits, labels)
        return (loss, outputs) if return_outputs else loss

# 12. TRAINER INIT
trainer = WeightedTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized["train"],
    eval_dataset=tokenized["validation"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=2)],
)

# 13. TRAIN
print("\n==============================")
print("🚀 BẮT ĐẦU TRAIN ALBERT-base")
print("==============================\n")
trainer.train(resume_from_checkpoint=last_checkpoint)

manage_checkpoints(OUTPUT_DIR, keep_latest=2)

# 14. TEST
print("\n🎯 ĐÁNH GIÁ TRÊN TEST SET")
results = trainer.evaluate(tokenized["test"])
for k, v in results.items():
    print(f"{k}: {v}")

# 15. SAVE MODEL
final_model_path = os.path.join(OUTPUT_DIR, "final_best_model")
trainer.save_model(final_model_path)
tokenizer.save_pretrained(final_model_path)
print("\n✅ ĐÃ LƯU MÔ HÌNH TẠI:\n", final_model_path)

# CLEAN UP
for f in os.listdir(OUTPUT_DIR):
    if f.startswith("checkpoint-") or f == "runs":
        shutil.rmtree(os.path.join(OUTPUT_DIR, f), ignore_errors=True)

print("\n🧹 Dọn dẹp hoàn tất!")


Device: Tesla T4 | CUDA: True | VRAM: 15.8 GB
Mounted at /content/drive
Drive: 44.8GB / 120.9GB (37.0%)
Checkpoint gần nhất: Không có → train từ đầu

Tải dataset WELFake...


README.md: 0.00B [00:00, ?B/s]

data/train-00000-of-00001-290868f0a36350(…):   0%|          | 0.00/152M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/72134 [00:00<?, ? examples/s]

Số mẫu ban đầu: 72134
Sau xử lý: 63323
Fake=54.5%, Real=45.5%
Class weights: [0.9172195  1.09920497]
Train=47492, Val=7915, Test=7916


...


🚀 BẮT ĐẦU TRAIN ALBERT-base



Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1,Auc
1,0.167300,0.190303,0.942893,0.947115,0.942893,0.943010,0.990249


Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1,Auc
1,0.167300,0.190303,0.942893,0.947115,0.942893,0.943010,0.990249
2,0.118800,0.099241,0.965888,0.966109,0.965888,0.965914,0.994698
3,0.074500,0.148381,0.969299,0.969663,0.969299,0.969329,0.995634



🎯 ĐÁNH GIÁ TRÊN TEST SET


eval_loss: 0.10942349582910538
eval_accuracy: 0.9768822637695806
eval_precision: 0.977046011700241
eval_recall: 0.9768822637695806
eval_f1: 0.9768979320317633
eval_auc: 0.997202141931091
eval_runtime: 76.8431
eval_samples_per_second: 103.015
eval_steps_per_second: 6.442
epoch: 3.0

✅ ĐÃ LƯU MÔ HÌNH TẠI:
 /content/drive/MyDrive/WELFake_ALBERT_base_Pro/final_best_model

🧹 Dọn dẹp hoàn tất!
